##Preprocess dataset


In [ ]:
import os
import requests

api_url = "https://api.github.com/repos/{owner}/{repo}/contents/{path}"
owner = "smartbugs"
repo = "smartbugs-curated"

In [ ]:
def download_content(node):
  content_url = node["download_url"]
  content_response = requests.get(content_url)
  file_path = f"downloaded_files/{node['path']}"

  # Check if the directory exists, and create it if it doesn't
  directory = os.path.dirname(file_path)
  if not os.path.exists(directory):
    os.makedirs(directory)
  
  if content_response.status_code == 200:
      with open(f"downloaded_files/{node['path']}", "wb") as f:
          f.write(content_response.content)
  else:
      print(f"Error downloading {node['path']}: {content_response.status_code}")

In [ ]:
def traverse_tree(node):

  if node['type'] == 'dir':
    call_response(node['path'])
  else:
    if node["name"].endswith(".sol"):
      download_content(node)

  

In [ ]:
def call_response(folder_path):

  url = api_url.format(owner=owner, repo=repo, path=folder_path)
  response = requests.get(url)

  if response.status_code == 200:
    os.makedirs("downloaded_files", exist_ok=True)
    for node in response.json():
      traverse_tree(node)
  else:
    print(f"Error listing files in folder: {response.status_code}")

  return


In [ ]:
call_response("dataset")

In [ ]:
folder_path = 'downloaded_files/dataset'
number_of_contracts = 0
contract_mapping = {}
contract_paths = []

for control in os.listdir(folder_path):
  control_folder_path = os.path.join(folder_path, control)
  
  for vulnerable_contract in os.listdir(control_folder_path):
    contract_path = os.path.join(control_folder_path, vulnerable_contract)
    contract_paths.append(contract_path)
    number_of_contracts += 1
    contract_mapping[vulnerable_contract] = control

print('Number of contracts: ' + str(number_of_contracts))

Number of contracts: 143


## Slither
Run and test out slither on 143 contracts

In [ ]:
#set up slither
!pip install slither-analyzer
!pip install solc-select
!solc-select install 0.4.24
import os
import subprocess
os.environ['SOLC_VERSION'] = '0.4.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 655.6/655.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.0/572.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
# #analyse all the contracts using slither
print("\n=================Running Slither==========================\n")
slither_result = {}
for contract_path in contract_paths:
  try:
    p = "/content/" + contract_path
    result = subprocess.run(['slither', p], capture_output=True, text=True)
    slither_result[contract_path] = result.stderr
  except:
    print('unable to run slither for ' + contract_path)


=================Running Slither==========================



In [ ]:
# Open the file for writing
with open('slither_analysis_result.json', 'w') as f:
    # Convert the dictionary to JSON and write it to the file
    json.dump(slither_result, f)

## Mythril
Run and test out Mythrill on all the 143 contracts

In [ ]:
!pip install mythril

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.0/481.0 kB 40.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━

In [ ]:
p = contract_paths[0]
result = subprocess.run(['myth','analyze', p], capture_output=True, text=True)
print(result)

CompletedProcess(args=['myth', 'analyze', 'downloaded_files/dataset/front_running/odds_and_evens.sol'], returncode=1, stdout="==== Unchecked return value from external call. ====\nSWC ID: 104\nSeverity: Medium\nContract: OddsAndEvens\nFunction name: getProfit()\nPC address: 428\nEstimated Gas Usage: 2505 - 37116\nThe return value of a message call is not checked.\nExternal calls return a boolean value. If the callee halts with an exception, 'false' is returned and execution continues in the caller. The caller should check whether an exception happened and react accordingly to avoid unexpected behavior. For example it is often desirable to wrap external calls in require() so the transaction is reverted if the call fails.\n--------------------\nIn file: downloaded_files/dataset/front_running/odds_and_evens.sol:50\n\nmsg.sender.send(this.balance)\n\n--------------------\nInitial State:\n\nAccount: [CREATOR], balance: 0xf00b80000040002, nonce:0, storage:{}\nAccount: [ATTACKER], balance: 0x

In [ ]:
# #analyse all the contracts using mythrill
print("\n\n\n\=================Running Mythrill==========================")
mythrill_result = {}
cnt = 0
for contract_path in contract_paths:
  cnt +=1
  print(cnt)

  try:
    p = contract_path
    result = subprocess.run(['myth','analyze', p], capture_output=True, text=True)
    mythrill_result[contract_path] = result.stdout
  except Exception as e:
    print(e)
    print('unable to run mythrill for ' + contract_path)




\=================Running Mythrill==========================
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [ ]:
print(len(mythrill_result))

In [ ]:
# # Open the file for writing
# with open('mythrill_analysis_run_10_result.json', 'w') as f:
#     # Convert the dictionary to JSON and write it to the file
#     json.dump(mythrill_result, f)